### This jupyternotebook is used to join openRouteServices csv files


**Note**
- Most code are used to join and save the data
- Running this notebook will not return all data 
- The purpose of this notebook is simply to serve as a hub to join files


In [17]:
import pandas as pd
import numpy as np
import os

part_1 = pd.read_csv("../data/raw/domain_and_distance_[5332, 5828].csv")
part_2 = pd.read_csv("../data/raw/domain_and_distance_[5828, 7156].csv")
part_3 = pd.read_csv("../data/raw/domain_and_distance_[7156, 7244].csv")
part_4 = pd.read_csv("../data/raw/domain_and_distance_[7244, 7998].csv")


In [18]:
part_2.drop(len(part_2)-1, inplace=True)

In [19]:
part_3.drop(len(part_3)-1, inplace=True)
part_3.drop(0, inplace=True)

In [20]:
part_4.drop(0, inplace=True)

In [21]:
result = pd.concat([part_1, part_2, part_3, part_4], axis=0, ignore_index=True)

In [22]:
# Save the concatenated DataFrame to CSV
result.to_csv('../data/raw/combined_domain_distance_part_3.csv', index=False)

In [23]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [24]:
import pandas as pd
import numpy as np
import os


In [25]:
part_1 = pd.read_csv("../data/raw/combined_domain_distance_part_1.csv")
part_2 = pd.read_csv("../data/raw/combined_domain_distance_part_2.csv")
part_3 = pd.read_csv("../data/raw/combined_domain_distance_part_3.csv")
part_4 = pd.read_csv("../data/raw/combined_domain_distance_part_4.csv")
part_5 = pd.read_csv("../data/raw/combined_domain_distance_part_5.csv")

In [26]:
amenity_pd = pd.concat([part_1, part_2, part_3, part_4, part_5], axis=0, ignore_index=True)

In [27]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import json
import re
import pandas as pd
import numpy as np
import requests


# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Open Route Services")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.port", "6066")
    .getOrCreate()
)

In [28]:
cleaned_domain_data = spark.read.parquet("../data/raw/cleaned_domain_current_listings.parquet")

In [29]:
cleaned_domain_data.count()

13328

In [30]:
amenity_pd.shape

(13328, 19)

In [31]:
domain_pd_df = cleaned_domain_data.toPandas()

In [32]:
amenity_pd.drop('House Name', axis=1, inplace=True)

In [33]:
domain_joined_distance = pd.concat([domain_pd_df, amenity_pd], axis=1)

In [34]:
spark_df = spark.createDataFrame(domain_joined_distance)

In [35]:
spark_df.count()

13328

In [36]:
len(spark_df.columns)

30

In [37]:
spark_df.tail(5)

[Row(bedrooms=1, bathrooms=1, car_parks=0, url='https://www.domain.com.au/1111-228-la-trobe-street-melbourne-vic-3000-17194010\n', name='1111/228 La Trobe Street, Melbourne VIC 3000', cost_text='$660 per week', type='Apartment / Unit / Flat', latitude='-37.809615', longitude='144.9623021', suburb=' Melbourne VIC 3000', postcode='3000', rent_pw=660, school_name='Ozford College', school_walking=9.3, school_driving=9.3, hospital_name='Peter MacCallum Cancer Centre', hospital_walking=28.42, hospital_driving=34.4, doctors_name='Breast Imaging Victoria', doctors_walking=11.5, doctors_driving=11.5, bank_name='BOM', bank_walking=6.18, bank_driving=11.61, restaurant_name='Vegie Tribe', restaurant_walking=8.07, restaurant_driving=8.07, PTV_name='Flinders Street Railway Station (Melbourne City)', PTV_walking=6.42, PTV_driving=61.66),
 Row(bedrooms=1, bathrooms=1, car_parks=0, url='https://www.domain.com.au/12-246-albert-street-east-melbourne-vic-3002-17196721\n', name='12/246 Albert Street, East 

In [38]:
# write to raw layer
import os

output_directory = '../data/raw/domain_listing_openRouteServices'
os.makedirs(output_directory, exist_ok=True)

spark_df.write.mode('overwrite').parquet(output_directory)

24/10/04 01:48:56 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [39]:
spark_df.limit(5)

bedrooms,bathrooms,car_parks,url,name,cost_text,type,latitude,longitude,suburb,postcode,rent_pw,school_name,school_walking,school_driving,hospital_name,hospital_walking,hospital_driving,doctors_name,doctors_walking,doctors_driving,bank_name,bank_walking,bank_driving,restaurant_name,restaurant_walking,restaurant_driving,PTV_name,PTV_walking,PTV_driving
2,1,1,https://www.domai...,705/8 Marmion Pla...,$600 per week,Apartment / Unit ...,-37.8134708,144.9424794,Docklands VIC 3008,3008,600,Ozford College,9.3,9.3,Peter MacCallum C...,28.42,34.4,Victoria Harbour ...,5.01,14.84,Westpac,6.26,18.28,Achelya,7.33,13.4,North Melbourne R...,6.85,6.85
3,2,2,https://www.domai...,5/18-20 Ibbottson...,$650 Per Week,Townhouse,-37.70987239999999,145.0844928,Watsonia VIC 3087,3087,650,The Austin School,38.22,38.22,Mercy Hospital fo...,22.44,34.99,Dynamic Health Os...,12.95,24.32,BOM,5.51,16.99,The A Team Kitchen,8.42,21.83,Broadmeadows Rail...,3.27,37.47
3,2,1,https://www.domai...,2109/35 Malcolm S...,"$1,150/week",Apartment / Unit ...,-37.8369683,144.9964622,South Yarra VIC ...,3141,1150,Holmes Grammar Sc...,12.57,17.05,Epworth Richmond ...,10.78,10.78,BB Skin Boutique,0.58,4.86,HSBC,5.14,15.74,Hunky Dory Fish &...,5.34,14.4,Richmond Railway ...,1.64,45.69
2,1,1,https://www.domai...,4/17a The Esplana...,$475.00 per week,Apartment / Unit ...,-38.1345686,144.3548803,Geelong VIC 3220,3220,475,Ozford College,9.3,9.3,Boardwalk Health ...,5.99,10.23,Dentists & Doctors,11.21,11.21,Bendigo Bank,5.74,14.96,Tulip,8.92,18.15,Geelong Railway S...,1.6,29.94
3,2,2,https://www.domai...,501/446 Malvern R...,$2200 Per Week,Apartment / Unit ...,-37.8479885,145.0012197,Prahran VIC 3181,3181,2200,Holmes Grammar Sc...,12.57,17.05,Epworth Richmond ...,10.78,10.78,High Street Medic...,8.01,14.89,ANZ,2.73,11.05,Bistro Thierry,8.43,17.89,Richmond Railway ...,1.64,45.69


In [40]:
school = pd.read_csv("../data/raw/schools.csv")
school.head(5)

,Unnamed: 0,x,y,school_name,number_vce_subjects,satisfactory_complete_vce_percent,median_study_score,study_score_over_40_percent,percentage_applying_to_victorian_uni
0,0,146.96093,-36.73310,bright p college,30.0,95.00000,29.000000,4.800000,20.000000
1,1,144.16067,-38.04200,bannockburn p college,33.0,92.00000,24.000000,0.900000,42.000000
2,2,144.11368,-36.04736,pyramid hill college,3.0,95.62892,28.756957,7.186515,67.054348
3,3,149.75430,-37.55943,mallacoota p college,7.0,100.00000,34.000000,16.700000,50.000000
4,4,149.15302,-37.56970,cann river p college,8.0,100.00000,24.000000,7.186515,67.000000
